In [ ]:
from AFEP_parse import *

from IPython.display import display, Markdown

import warnings #Suppress future warnings from pandas.
warnings.simplefilter(action='ignore', category=FutureWarning)

# How to Use this Notebook:

## This notebook is specially written for the SAFEP tutorial. 
For more up-to-date and general versions see the SAFEP github.

## This and other SAFEP notebooks make use of pyMBAR and Alchemlyb. 
For more information see Shirts and Chodera (2008), "Statistically optimal analysis of samples from multiple equilibrium states" doi: 10.1063/1.2978177

## User Parameters:
**IMPORTANT: Make sure the temperature matches the temperature used to run the simulations.** <br>
The decorrelate flag should be set to True for general use. <br>
The detectEQ (automatic equilibrium detection) flag is optional. If you have very poor convergence but believe your system is well-sampled, you may wish to try it.

In [ ]:
temperature = 300
radius = 3 #Radius of the spherical restraint used for the DBC TI calculation

decorrelate = True #Flag for automatic decorrelation
detectEQ = False #Flag for automatic equilibrium detection
RT = 0.00198720650096 * temperature # RT in kcal/mol

root = '..' #Root path to fepout files

***
# Process the Bound Data

In [ ]:
path=f'{root}/AFEP-Bound-Decoupling/output/' #Path to fepout files
filename='*.fepout' #This can be a regex pattern if you have more than one fepout file
#Caution: if you have multiple fepout files, name them alphanumerically

In [ ]:
fepoutFiles = glob(path+filename) #Resolve any naming regular expressions
u_nk, _ = readAndProcess(fepoutFiles, temperature, decorrelate, detectEQ) #u_nk stores the fep data
perWindow, cumulative = doEstimation(u_nk) #Run the BAR estimator on the fep data
forward, forward_error, backward, backward_error = doConvergence(u_nk) #Used later in the convergence plot

dG_site = np.round(cumulative.BAR.f.iloc[-1]*RT, 1)
error_site = np.round(cumulative.BAR.errors.iloc[-1]*RT, 1)
changeAndError = f'\u0394G<sub>site</sub> = {dG_site}\u00B1{error_site} kcal/mol'
Markdown('<font size=7>{}</font><br/>'.format(changeAndError))

In [ ]:
#Make some general plots
width = 8
height = 4
fig, axes = plotGeneral(cumulative, perWindow, RT, width=width, height=height, PDFtype='Histogram')
plt.savefig(f'{path}_bound_generalFigures.pdf')
plt.show()

fig, convAx = plt.subplots(1,1)
convAx = convergencePlot(convAx, forward*RT, forward_error*RT, backward*RT, backward_error*RT)

fig.set_figwidth(width*3/4)
fig.set_figheight(height)

plt.savefig(f'{path}_bound_convergence.pdf')
plt.show()

***
# Process the Unbound Data

In [ ]:
path=f'{root}/AFEP-Hydration/'
filename='*.fepout' #This can be a regex pattern if you have more than one fepout file

fepoutFiles = glob(path+filename)


In [ ]:
fepoutFiles = glob(path+filename) #Resolve any naming regular expressions
u_nk, _ = readAndProcess(fepoutFiles, temperature, decorrelate, detectEQ) #u_nk stores the fep data
perWindow, cumulative = doEstimation(u_nk) #Run the BAR estimator on the fep data
forward, forward_error, backward, backward_error = doConvergence(u_nk) #Used later in the convergence plot

dG_site = np.round(cumulative.BAR.f.iloc[-1]*RT, 1)
error_site = np.round(cumulative.BAR.errors.iloc[-1]*RT, 1)
changeAndError = f'\u0394G<sub>bulk</sub> = {dG_site}\u00B1{error_site} kcal/mol'
Markdown('<font size=7>{}</font><br/>'.format(changeAndError))

In [ ]:
width = 8
height = 4
fig, axes = plotGeneral(cumulative, perWindow, RT, width=width, height=height, PDFtype='PDF')
plt.savefig(f'{path}_bound_generalFigures.pdf')
#plt.savefig(f'{path}dG_{affix}.png', dpi=600)
plt.show()

fig, convAx = plt.subplots(1,1)
convAx = convergencePlot(convAx, forward*RT, forward_error*RT, backward*RT, backward_error*RT)
convAx.set(xlabel='Fraction of Simulation Time', ylabel=r'Total $\rm\Delta G_{\lambda}$ (kcal/mol)')
fig.set_figwidth(width*3/4)
fig.set_figheight(height)
convAx.legend()
plt.savefig(f'{path}_bound_convergence.pdf')
plt.show()

***
# Process the DBC TI calculation

In [ ]:
def plotTI(cumulative, perWindow, width=8, height=4, PDFtype='KDE', hystLim=(-1,1), fwdColor='#0072B2', bwdColor='#D55E00'):
    fig, ((cumAx, del1),( eachAx, del2),(hystAx, pdfAx)) = plt.subplots(3,2, sharex='col', sharey='row', gridspec_kw={'width_ratios': [2, 1]})

    fig.delaxes(del1)
    fig.delaxes(del2)

    # Cumulative change in kcal/mol
    cumAx.plot(cumulative.index, cumulative['mean'], marker=None, linewidth=1, color='black', label='mean')
    cumAx.plot(cumulative.index, cumulative.ff, marker=None, linewidth=1, alpha=0.5, color=fwdColor, label='forward samples')
    cumAx.plot(cumulative.index, cumulative.fb, marker=None, linewidth=1, alpha=0.5, color=bwdColor, label='backward samples')
    finalEstimate = cumulative['mean'][1]
    cumAx.axhline(finalEstimate, linestyle='-', color='gray', label=f'Final Value:\n{np.round(finalEstimate,1)}kcal/mol')
    cumAx.legend()                  
    cumAx.set(ylabel=r'Cumulative $\rm\Delta G_{\lambda}$'+'\n(kcal/mol)')

    # Per-window change in kcal/mol
    eachAx.plot(perWindow.index, perWindow.dG_f, marker=None, linewidth=1, alpha=0.5, color=fwdColor)
    eachAx.plot(perWindow.index, -perWindow.dG_b, marker=None, linewidth=1, alpha=0.5, color=bwdColor)
    eachAx.plot(lmid, np.mean([perWindow.dG_f,-perWindow.dG_b], axis=0), linestyle='-.', color='black')
    eachAx.set(ylabel=r'$\rm\Delta G_{\lambda}$'+'\n(kcal/mol)')

    
    #Hysteresis Plots
    diff = perWindow.difference
    hystAx.vlines(perWindow.index, np.zeros(len(perWindow)), diff, label="fwd - bwd", linewidth=2)
    hystAx.set(xlabel=r'$\lambda$', ylabel=r'$\delta_\lambda$ (kcal/mol)', ylim=hystLim)
    
    if PDFtype=='KDE':
        kernel = sp.stats.gaussian_kde(diff)
        pdfX = np.linspace(-1, 1, 1000)
        pdfY = kernel(pdfX)
        pdfAx.plot(pdfY, pdfX, label='KDE')
    elif PDFtype=='PDF':
        pdfX, pdfY = np.histogram(X, nbins=20)
        pdfAx.plot(pdfY, pdfX,  label="Estimated Distribution")
    else:
        raise(f"Error: PDFtype {PDFtype} not recognized")
    
    pdfAx.set(xlabel=PDFtype)

    std = np.std(diff)
    mean = np.average(diff)
    temp = pd.Series(pdfY, index=pdfX)
    mode = temp.idxmax()
    
    textstr = r"$\rm mode=$"+f"{np.round(mode,2)}"+"\n"+fr"$\mu$={np.round(mean,2)}"+"\n"+fr"$\sigma$={np.round(std,2)}"
    props = dict(boxstyle='square', facecolor='white', alpha=0.5)
    pdfAx.text(0.15, 0.95, textstr, transform=pdfAx.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)

    fig.set_figwidth(width)
    fig.set_figheight(height*3)
    fig.tight_layout()
    
    return fig, [cumAx,eachAx,hystAx,pdfAx] 

In [ ]:
def makedUFrame(windows=11, samples=10, timeFrequency=100, precision=3, first=0):
    #Make an empty data frame
    lambdas = np.round(np.linspace(0,1,windows), precision)
    dL = np.round(lambdas[1]-lambdas[0], precision)
    times = first+(timeFrequency*np.arange(samples)).astype(int) #steps are integers
    indcs = pd.MultiIndex.from_product([times, lambdas], names = ['time','from-lambda']) #Fully cross the FEPs and the steps like you'd have in raw data (from NAMD at least)
    data = pd.DataFrame(columns=lambdas, index=indcs) #build the dataframe
    data.sort_index(level=1) #make it pretty
    
    return data, dL




def get_TI(dU_Frame):
    #the data frame is organized from index level 1 (fep-lambda) TO column
    #dG will be FROM column TO index
    groups = dU_Frame.groupby(level=1)
    dG=pd.DataFrame([])
    for name, group in groups:
        dG[name] = np.mean(group)

    dG_f=np.diag(dG, k=1)
    dG_b=np.diag(dG, k=-1)

    l=dG.columns.to_list()
    l_mid = np.mean([l[1:],l[:-1]], axis=0)

    return l, l_mid, dG_f, dG_b


def doTI(dU_Frame):
    cumulative = pd.DataFrame()
    perWindow = pd.DataFrame()
        
    l, lmid, dG_fs, dG_bs = get_TI(dU_Frame)

    cumulative['ff'] = np.insert(np.cumsum(dG_fs),0,0)
    cumulative['fb'] = np.insert(-np.cumsum(dG_bs),0,0)
    cumulative['mean'] = np.mean(cumulative, axis=1) 
    cumulative.index = l 

    perWindow['dG_f'] = dG_fs
    perWindow['dG_b'] = dG_bs
    perWindow['difference'] = np.array(dG_fs)+np.array(dG_bs)        
    perWindow.index = lmid
    
    return perWindow.copy(), cumulative.copy()

In [ ]:
def makeHarmonicWall(FC=10, targetFC=0, targetFE=1, upperWalls=1, schedule=None, numSteps=1000, targetEQ=500, name='HW', lowerWalls=None):
    HW = {'name':name, 'targetFC':targetFC, 'targetFE':targetFE, 'FC':FC, 'upperWalls':upperWalls, 'schedule':schedule, 'numSteps':numSteps, 'targetEQ':targetEQ, 'lowerWalls':lowerWalls}
    return HW

def HW_U(HW, coord, L):
    d=0
    if HW['upperWalls'] and coord>HW['upperWalls']:
        d = coord-HW['upperWalls']
    elif HW['lowerWalls'] and coord<HW['lowerWalls']:
        d = coord-HW['lowerWalls']
    
    if d!=0:
        dk = HW['targetFC']-HW['FC']
        la = L**HW['targetFE']
        kL = HW['FC']+la*dk
        U = 0.5*kL*(d**2)
    else:
        U=0
    return U

def dHW_U(HW, coord, L):
    d=0
    if HW['upperWalls'] and coord>HW['upperWalls']:
        d = coord-HW['upperWalls']
    elif HW['lowerWalls'] and coord<HW['lowerWalls']:
        d = coord-HW['lowerWalls']
    
    if d!=0:
        dk = HW['targetFC']-HW['FC']
        la = L**HW['targetFE']
        kL = HW['FC']+la*dk
        U = 0.5*kL*(d**2)
    else:
        U=0
    return U

In [ ]:
def fill_dUFrame(dU_Frame, dataTI, dL, step):
    for key, group in dataTI.groupby('L'):
        current = np.nan
        forward = np.nan
        backward = np.nan

        current = np.array([HW_U(DBC, coord, key) for coord in group.DBC])

        dU_Frame.loc[(slice(None), key), key] = 0


        nextKey = np.round(key+dL, 3)
        lastKey = np.round(key-dL, 3)
        if key < 1:
            forward = np.array([HW_U(DBC, coord, nextKey) for coord in group.DBC])

            dUf = forward-current
            dU_Frame.loc[(slice(0,len(dUf)*step-1), key), nextKey] = dUf
            #group.loc[:,nextKey]=dUf
        if key > 0:
            backward = np.array([HW_U(DBC, coord, lastKey) for coord in group.DBC])
            #U_Frame.loc[(slice(None), key), lastKey] = backward   
            dUb = backward - current
            dU_Frame.loc[(slice(0,len(dUb)*step-1), key), lastKey] = dUb
            #group.loc[:,lastKey]=dUf

    #dU_frame = pd.concat(rebuild)
    dU_Frame = dU_Frame.sort_index(level=1).sort_index(axis=1)
    dU_Frame = dU_Frame.dropna(how='all', axis=0)
    #U_Frame = U_Frame.sort_index(level=1).sort_index(axis=1)
    return dU_Frame.copy()

In [ ]:

step = 100
path = f'{root}/RFEP/output/'
#path = '/home/ems363/Documents/RFEP/'
dataTI = pd.read_csv(glob(path+'*.colvars.traj')[0], delim_whitespace=True, names=['step','DBC', 'distance', 'E_dist', 'E_DBC'], comment='#', index_col=0)
dataTI = dataTI[dataTI.index>=1000][1:]
dataTI.index = dataTI.index-1000

#For comparison to current method. Not needed in release form.
RFEPdat = pd.read_csv(glob(path+'RFEP.dat')[0], delim_whitespace=True, names=['L', 'dGperL'], comment='#', index_col=0)

Lsched = np.linspace(1,0,41)

Dist = makeHarmonicWall(FC=200, upperWalls=radius, name='Dist_rest')
DBC = makeHarmonicWall(FC=0, targetFC=200, targetFE=6, upperWalls=1.5, targetEQ=500, numSteps=300000, name='DBC_rest', schedule=Lsched)
Ls = (dataTI.index.values-1)//DBC['numSteps']
Ls[0] = 0

In [ ]:
Ls[Ls==41] = 40 #Fix in case there are extra samples for the last window

In [ ]:
dataLs = np.round([DBC['schedule'][i] for i in Ls], 3)
dataTI.loc[:,'L'] = dataLs
dataTI = dataTI.iloc[1:]

In [ ]:
if np.isnan(dataTI.E_dist.iloc[0]):
    dataTI.loc[:,'E_dist'] = [HW_U(Dist, coord, 0) for coord in dataTI.distance]
if np.isnan(dataTI.E_DBC.iloc[0]):
    dataTI.loc[:,'E_DBC'] = [HW_U(DBC, coord, L) for coord, L in zip(dataTI.DBC, dataTI.L)]

In [ ]:
plt.plot(dataTI.E_dist, label='spherical restraint', alpha=0.5)
plt.plot(dataTI.E_DBC, label='DBC restraint', alpha=0.5)
plt.legend()
plt.savefig(f'{path}_restraint_overlap.pdf')

In [ ]:
dU_Frame, dL = makedUFrame(41, 4000, timeFrequency=step)
dU_Frame = fill_dUFrame(dU_Frame, dataTI, dL, step)
equilibrated = dU_Frame[dU_Frame.index.get_level_values(0) > DBC['targetEQ']]
l, lmid, Gf, Gb = get_TI(equilibrated)
TIperWindow, TIcumulative = doTI(equilibrated)

In [ ]:
dG_DBC = np.round(TIcumulative['mean'][1], 1)
error_DBC = np.round(TIcumulative['mean'][1]-TIcumulative['ff'][1], 1)
changeAndError = f'\u0394G<sub>DBC</sub> = {dG_DBC}\u00B1{error_DBC} kcal/mol'
Markdown('<font size=7>{}</font><br/>'.format(changeAndError))

In [ ]:
fwdColor='#0072B2'
bwdColor='#D55E00'
plt.plot(RFEPdat*-1, label='colvars estimate', color='red')
plt.plot(lmid, Gf, label='forward estimate', color=fwdColor)
plt.plot(lmid, -Gb, label='backward estimate', color=bwdColor)
plt.plot(lmid, np.mean([Gf,-Gb], axis=0), label='fwd-bwd mean', linestyle='-.', color='black')
plt.savefig(f'{path}_TI_vs_colvarEst.pdf')
plt.legend()

In [ ]:
fig, axes = plotTI(TIcumulative, TIperWindow, hystLim=(-0.25, 0.25))
plt.savefig(f'{path}_TI_general.pdf')
plt.show()

***
# Calculate the Volumetric Restraint Contribution

In [ ]:
molar = 1660 #cubic angstroms per particle in a one molar solution
dG_V = np.round(-RT*np.log(4/3*sp.pi*radius**3/molar), 1)
error_V = 0

In [ ]:
dG_V

***
# Compute the final estimate

In [ ]:
dG_binding = dG_bulk + dG_V - dG_DBC - dG_site
error_binding = np.sqrt(np.mean(np.array([error_bulk, error_V, error_site, error_DBC])**2))

In [ ]:
print(f'\u0394G = {np.round(dG_binding, 1)}\u00B1{np.round(error_binding, 1)} kcal/mol')